In [9]:
from math import *
from vpython import *
from numpy.random import randint
import random

#Defining a Function to Make Python Solve Stuff
def my_function(c, x, y):
    return (-1/(c - y)**2+2*y/(x*x + y*y)**1.5)

def dx(fn, c, x, y, delta=0.001): #x represents sun0_dist and c represents sunfar_dist. this returns the value of 
    return (fn(c, x, y+delta) - fn(c, x, y))/delta

def solve(fn, c, x, value, y, maxtries=10000, maxerr=0.00000001):
    for tries in range(maxtries):
        err = fn(c, x, y) - value
        if abs(err) < maxerr:
            return y
        slope = dx(fn, c, x, y)
        y -= err/slope
    raise ValueError('no solution found')



#
# Needed Constants
#
#origin_distance = 149597870000*10
sun0_dist = 149597870000*4
sunfar_dist = 149597870000*10
speed_of_earth = 0

out_of_range = True
while out_of_range == True:
    # y = solve(my_function, sunfar_dist, sun0_dist, 0, randint(sunfar_dist*.1, sunfar_dist*.9))
    y = solve(my_function, sunfar_dist, sun0_dist, 0, sunfar_dist*random.random())

    if 0 <= y < sunfar_dist:
        out_of_range = False
        print('Lagrange Point: ', y)

# Based on Earth's lowest and highest recorded temps
min_temp = 2.7
max_temp = 4273.15
luminosity = 38*10**25

# Use earth's albedo: A = 0.31
A = 0.31
#
# Set up the displays
#
scene2 = canvas(title='Earth orbiting the Sun',caption='Animated Display',
     center=vector(0,0,0), background=color.black)
#
# Make the radius of each object large enough to see them
#

earth = sphere(pos=vector(0,y,0),radius=1e11,color=color.green)
suns = [sphere(pos=vector(sun0_dist,0,0),radius=1e11,color=color.yellow), sphere(pos=vector(-sun0_dist,0,0),radius=1e11,color=color.yellow), sphere(pos=vector(0,sunfar_dist,0),radius=1e11,color=color.yellow)]
sunMass = 1.989e30
for sun in suns:
    sun.mass = sunMass
suns[0].momentum = vector(0,16000,0)*suns[0].mass
suns[1].momentum = vector(0,16000,0)*suns[1].mass
suns[2].momentum = vector(0,16000,0)*suns[2].mass

#
G = 6.6743e-11
#
earth.mass = 5.972e24
#
#
# Create a trail for the earth, and vectors for the force on the earth.
# scale should be a number that lets is see the force arrow on the plot.
#
earth.trail = curve(color=color.magenta)
earth.trail.append(pos=earth.pos)

totalMass = 3*sun.mass + earth.mass
#
# COM of suns and earth
Rcm = (suns[0].pos*sun.mass + suns[1].pos*sun.mass + suns[2].pos*sun.mass + earth.pos*earth.mass)/(totalMass)
#
#
scale = 3e-13

earth.totalTemp = 0
totalInitialSunForce = vector(0,0,0)
#want to find minimum starting velocity
for sun in suns:
    sun.initialforce = vector(0,0,0)
    sun.initialforce += norm(sun.pos-earth.pos)* -(G* earth.mass * sun.mass / mag(sun.pos-earth.pos)**2)
    totalInitialSunForce += sun.initialforce
minimum_radius = (luminosity*(1-A)/(16*(773.15)**4))**0.5
initial_speed = (abs(mag(earth.pos-Rcm) * (comp(totalInitialSunForce,Rcm))/earth.mass))**.5
print(initial_speed)
print(sun.initialforce)
print(totalInitialSunForce)
        
earth.momentum = vector(0,0*(initial_speed), 0)*earth.mass
# 
# Initial time is 0, and the time step is twelve hours
#
time = 0
dt =  5*3.154e+4
# dt = 3600
# 
# Runs until temperature is out of range or hits a sun
#
# x = True
x=False
earth.temp = 0

while x:
    #print(x)
    #rate(200000)
#
    
    days = time/(3600*24)
    speed_of_earth = 0
    earth.totalTemp = 0
    earth.temp = 0
#
#  Compute the force on the earth using our force function.
#
    earth.force = vector(0,0,0)
    for sun in suns:
        earth.force += norm(earth.pos-sun.pos)* -(G* earth.mass * sun.mass / mag(earth.pos-sun.pos)**2)
#         earth.totalTemp += 279*(1-A)**(1/4)*(mag(earth.pos-sun.pos)*6.68459e-12)
#       Changed to +=
        earth.totalTemp += luminosity/(4*pi*mag(earth.pos-sun.pos)**2) * (1-A)/(4*567e-10)
        
    speed_of_earth = sqrt(mag(earth.force)*mag(Rcm)/earth.mass)
    earth.pos = earth.pos + (earth.momentum/earth.mass) * dt
    if time == 0 :
        print('initial speed:', speed_of_earth)
    earth.trail.append(pos=earth.pos)
    earth.temp = (earth.totalTemp)**(0.25)
    #print(earth.force)
    #print('temp now:', earth.temp)
    
# ???    
    for sun in suns:
        sun.force = vector(0,0,0)
        for sun2 in suns:
            if sun != sun2:
                sun.force += norm(sun.pos-sun2.pos)*-(G* sun.mass * sun2.mass / mag(sun.pos-sun2.pos)**2) 
        sun.force += norm(sun.pos-earth.pos)* -(G* earth.mass * sun.mass / mag(sun.pos-earth.pos)**2)
        sun.momentum = sun.momentum + sun.force * dt
        sun.pos = sun.pos + (sun.momentum/sun.mass) * dt

#         it won't crash into the sun? because of temp conditions

        if (mag(sun.pos-earth.pos) <= earth.radius ):
            print('Earth Crashed into One of the Suns.')
            x = False
#         if (earth.temp < 0 or earth.temp > 773.15):
#             print('Earth is too cold or hot.')
#             x = False
# THIS WILL ALWAYS BE ONE LOOP BEHIND
        if (earth.temp < min_temp):
            print('Too cold')
            x = False
        if (earth.temp > max_temp):
            print('Too hot.')
            x = False
    time += dt        
     

    
#re
print('years', (time/3.154e+7))
print('days',time/(3600*24))
print('speed', speed_of_earth)
print('temp', earth.temp)
print(earth.pos)
for sun in suns:
    print(sun.pos)

Lagrange Point:  638139405687.8152


<IPython.core.display.Javascript object>

3183.813414376612
<0, -1.07733e+21, 0>
<0, 4.34014e+20, 0>
years 0.0
days 0.0
speed 0
temp 0
<0, 6.38139e+11, 0>
<5.98391e+11, 0, 0>
<-5.98391e+11, 0, 0>
<0, 1.49598e+12, 0>


In [4]:
sphere()